# Evaluate Accuracy of the Oracle

We simulate the oracle based on the neutrality results of user studies in AAAI-14 paper. We know the percentages of neutral values per k-words documetns but not per snippet forms. 

Test: 
* Oracle accuracy
* Oracle neutrality

In [ ]:
DATA = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
import sys
import os
sys.path.append(os.path.abspath('C:/cygwin/home/mramire8/python_code/structured/'))
sys.path.append(os.path.abspath('/Users/maru/MyCode/structured/'))



In [5]:

import learner

from utilities.datautils import load_dataset
import utilities.experimentutils as exputil
import numpy as np
import experiment.base as exp
import nltk

# Read data (optionally from a pickled file if present)
import pickle


from time import time
import pickle


## Get the data ready
imdb_path = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
# imdb_path = '/Users/maru/MyCode/data/imdb'



In [ ]:
vct = exputil.get_vectorizer({'vectorizer':'bow', 'limit':None, 'min_size':2})

imdb =  load_dataset("imdb",imdb_path, keep_subject=True)
# sraa = load_dataset("aviation", 100, categories[0], vct2, 100, raw=True,  percent=.5, keep_subject=True)

imdb.train.bow = vct.fit_transform(imdb.train.data)



In [8]:
oracle = exputil.get_classifier('lrl1', parameter=0.3)
oracle.fit(imdb.train.bow, imdb.train.target)
imdb.test.bow = vct.transform(imdb.test.data)


In [9]:
import nltk
sent_tk = nltk.data.load('tokenizers/punkt/english.pickle')

d_sent = sent_tk.tokenize_sents(imdb.test.data)

In [154]:
from sklearn import metrics
from collections import defaultdict
def test_oracle(ora_clf, doc_sents, doc_y, sizes, vct):
    
    results = defaultdict(lambda: [])
    
    for sents,y in zip(doc_sents, doc_y):
        if len(sents) > 0:
            bow = vct.transform(sents)
            probs = ora_clf.predict_proba(bow)
            unc = probs.min(axis=1)
            preds = ora_clf.predict(bow)
            nonneu = unc < 0.4
            results['confidence'].extend(1-unc)
            acc = metrics.accuracy_score([y] *len(preds[nonneu]) ,preds[nonneu])
            if sum(nonneu)>0:
                results['accu'].append(acc)
            results['noneutrals'].append(1. * sum(nonneu) / len(nonneu) )
            results['size'].extend([len(s.split()) for s in sents])
            results['preds'].extend(preds[nonneu])
            results['trues'].extend([y] *len(preds[nonneu]))
    return results


In [155]:
res = test_oracle(oracle, d_sent, imdb.test.target, [1], vct)

In [156]:
import numpy as np 
from scipy import stats

def describe(v):
    n, min_max, mean, var, skew, kurt = stats.describe(v)
    return  "N:{},\tmin:{:>7.3f},\tmax:{:>7.3f},\tmean:{:>7.3f},\tstd.:{:>7.3f}".format(n, min_max[0], min_max[1], mean, var)

def print_describe(res):
    for k,v in res.items():
        print  "{:<15} {}".format(k,describe(v))
        
print_describe(res)


confidence      N:304952,	min:  0.500,	max:  1.000,	mean:  0.647,	std.:  0.013
preds           N:173726,	min:  0.000,	max:  1.000,	mean:  0.635,	std.:  0.232
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.603,	std.:  0.037
accu            N:24908,	min:  0.000,	max:  1.000,	mean:  0.748,	std.:  0.063
trues           N:173726,	min:  0.000,	max:  1.000,	mean:  0.522,	std.:  0.250
size            N:304952,	min:  1.000,	max:472.000,	mean: 18.554,	std.:176.596


In [157]:
print "Accuracy:", metrics.accuracy_score(res['trues'], res['preds'])

Accuracy: 0.736798176439


In [34]:
snip = exputil.get_tokenizer('snippet',snip_size=(2,2))
snip2=  snip.tokenize_sents(imdb.test.data)

In [158]:
res2 = test_oracle(oracle, snip2, imdb.test.target, [1], vct)
print_describe(res2)
print "Accuracy:", metrics.accuracy_score(res2['trues'], res2['preds'])

confidence      N:2176474,	min:  0.500,	max:  1.000,	mean:  0.696,	std.:  0.017
preds           N:1529325,	min:  0.000,	max:  1.000,	mean:  0.548,	std.:  0.248
noneutrals      N:24778,	min:  0.000,	max:  1.000,	mean:  0.753,	std.:  0.023
accu            N:24729,	min:  0.000,	max:  1.000,	mean:  0.793,	std.:  0.052
trues           N:1529325,	min:  0.000,	max:  1.000,	mean:  0.492,	std.:  0.250
size            N:2176474,	min:  2.000,	max:723.000,	mean: 36.672,	std.:357.528
Accuracy: 0.76253772089


In [160]:
1-0.647

0.353

In [82]:
exputil.get_tokenizer('snippet',snip_size=(1,3)).tokenize_sents([imdb.test.data[54]])

[[u'Bill Rebane\'s "The Capture of Bigfoot" is one of the most awful horror movies ever made.A greedy sawmill owner Harvey Olsen(Richard Kennedy)decides that he wants Bigfoot captured at all costs.However local game ranger Dave Garrett(Stafford Morgan)learns that the Bigfoot used to live in peace upset by a geological expedition,and sets out to protect the creature.There is nothing even remotely interesting in this piece of crap.The film is extremely dull and filled with horrible songs and cheap special effects.No gore,no suspense-just gigantic boredom.Avoid this horrible junk like the plague.']]

In [86]:
exputil.get_tokenizer('snippet',snip_size=(5,5)).tokenize_sents(['one two three'])

[['one two three']]

In [136]:
max((imdb.train.bow[0] + imdb.train.bow[1]) / (imdb.train.bow[0] + imdb.train.bow[1]))

matrix([[ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [137]:
print metrics.accuracy_score(imdb.test.target, oracle.predict(imdb.test.bow))


0.89519388531


In [139]:
print "Avg. s=1:", np.mean([len(ss) for s in d_sent])
print "Avg. s=2:", np.mean([len(s) for s in snip2])



Avg. s=1: 12.2034495178
Avg. s=2: 87.0972828044


In [146]:
s1= np.array([[len(c.split()) for c in r] for r in d_sent])

24989


In [149]:
s11 = []
for s in s1:
    for ss in s:
        s11.append(ss)

np.mean(s11)

18.553533015031874

In [152]:
um

ValueError: total size of new array must be unchanged

## Statistics of Snippet sizes 

Compute neutrality, accuracy of the oracle on first-1, 2, and 3 snippets. 

In [163]:
snipk = {}
for s in range(1,6):
    tk = exputil.get_tokenizer('first1snippet',snip_size=(s,s))
    data = tk.tokenize_sents(imdb.test.data)
    snipk[s] = test_oracle(oracle, data, imdb.test.target, [1], vct)
    print "== K = %s ===" % s
    print_describe(snipk[s])
    print "Accuracy:", metrics.accuracy_score(snipk[s]['trues'], snipk[s]['preds'])

== K = 1 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.668,	std.:  0.015
preds           N:15740,	min:  0.000,	max:  1.000,	mean:  0.659,	std.:  0.225
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.630,	std.:  0.233
accu            N:15740,	min:  0.000,	max:  1.000,	mean:  0.757,	std.:  0.184
trues           N:15740,	min:  0.000,	max:  1.000,	mean:  0.536,	std.:  0.249
size            N:24989,	min:  1.000,	max:472.000,	mean: 20.515,	std.:355.384
Accuracy: 0.756988564168
== K = 2 ===
confidence      N:24989,	min:  0.500,	max:  1.000,	mean:  0.729,	std.:  0.020
preds           N:19164,	min:  0.000,	max:  1.000,	mean:  0.581,	std.:  0.244
noneutrals      N:24989,	min:  0.000,	max:  1.000,	mean:  0.767,	std.:  0.179
accu            N:19164,	min:  0.000,	max:  1.000,	mean:  0.791,	std.:  0.165
trues           N:19164,	min:  0.000,	max:  1.000,	mean:  0.516,	std.:  0.250
size            N:24989,	min:  2.000,	max:723.000,	mean: 40.591,	std.:634.109
Accuracy: 0.7